In [5]:
import sys
sys.path.append('../') # or just install the module
sys.path.append('../../flaming-choripan') # or just install the module
sys.path.append('../../astro-lightcurves-handler') # or just install the module

In [6]:
from flamingchoripan.files import search_for_filedirs
from synthsne import C_

root_folder = '../save'
filedirs = search_for_filedirs(root_folder, fext=C_.EXT_SAMPLER)

────────────────────────────────────────────────────────────────────────────────────────────────────
found filedirs: (../save)
(0) - ../save/alerceZTFv7.1/survey-alerceZTFv7.1_bands-gr_mode-onlySNe/train/samplers.smplr - 9.655[mbs]
(1) - ../save/alerceZTFv7.1/survey-alerceZTFv7.1_bands-gr_mode-onlySNe/train+val/samplers.smplr - 10.244[mbs]
(2) - ../save/alerceZTFv7.1/survey-alerceZTFv7.1_bands-gr_mode-onlySNe_kfid-0/samplers.smplr - 7.858[mbs]
(3) - ../save/alerceZTFv7.1/survey-alerceZTFv7.1_bands-gr_mode-onlySNe_kfid-0/train/samplers.smplr - 7.597[mbs]
────────────────────────────────────────────────────────────────────────────────────────────────────
filedirs after searching with filters: (../save)
(0) - ../save/alerceZTFv7.1/survey-alerceZTFv7.1_bands-gr_mode-onlySNe/train/samplers.smplr - 9.655[mbs]
(1) - ../save/alerceZTFv7.1/survey-alerceZTFv7.1_bands-gr_mode-onlySNe/train+val/samplers.smplr - 10.244[mbs]
(2) - ../save/alerceZTFv7.1/survey-alerceZTFv7.1_bands-gr_mode-onlySNe_kfid

In [12]:
%load_ext autoreload
%autoreload 2
from synthsne import curve_statistics as cs

methods = ['linear', 'bspline', 'uniformprior', 'curvefit', 'mcmc']
#methods = ['curvefit', 'mcmc']
#methods = ['linear', 'bspline']
rootdir = '../save/alerceZTFv7.1/survey-alerceZTFv7.1_bands-gr_mode-onlySNe/train+val'
method = 'linear' # linear, bspline
cs.get_all_incorrects_fittings(rootdir, method)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


['ZTF19aawgopm']

In [17]:
%load_ext autoreload
%autoreload 2
from synthsne import curve_statistics as c

for method in methods:
    xe = cs.get_perf_times(rootdir, method)
    print(f'{method} ({len(xe)}): {xe}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
linear (502): .48±.09
bspline (0): ─
uniformprior (0): ─
curvefit (0): ─
mcmc (0): ─


In [42]:
%load_ext autoreload
%autoreload 2
from synthsne import curve_statistics as cs

method = 'linear'
rank, rank_bdict, band_names = cs.get_ranks(rootdir, method)
print(rank)
for b in band_names:
    print(rank_bdict[b])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
mb-rank[top10]:
(1) - ZTF18aaslhxt: 238.1672
(2) - ZTF18aaxkqgy: 181.2412
(3) - ZTF18aaxdrjn: 172.7882
(4) - ZTF18abukavn: 160.8303
(5) - ZTF18accnbgw: 152.9053
(6) - ZTF20aatzhhl: 146.8937
(7) - ZTF19acihlft: 140.8839
(8) - ZTF19aaknvuh: 133.0348
(9) - ZTF20abawgdl: 131.2031
(10) - ZTF19aaydpiq: 120.3180
g-rank[top10]:
(1) - ZTF18aaslhxt: 329.9000
(2) - ZTF18aaxkqgy: 287.8503
(3) - ZTF18accnbgw: 244.0280
(4) - ZTF19aaknvuh: 199.9777
(5) - ZTF18aaxdrjn: 190.0265
(6) - ZTF18abukavn: 189.9959
(7) - ZTF19aanxosu: 178.4678
(8) - ZTF20aaawbxl: 154.9424
(9) - ZTF20abawgdl: 148.1407
(10) - ZTF20aatzhhl: 146.8937
r-rank[top10]:
(1) - ZTF18abmxahs: 185.1511
(2) - ZTF18aaxdrjn: 155.5499
(3) - ZTF18abvptts: 152.2743
(4) - ZTF18aaslhxt: 146.4344
(5) - ZTF18aczrafz: 136.9509
(6) - ZTF18aarlpzd: 133.7569
(7) - ZTF18abukavn: 131.6647
(8) - ZTF19aampqcq: 129.9381
(9) - ZTF18aaxcntm: 126.8985
(10) - ZTF18aaumeys: 12

In [103]:
%load_ext autoreload
%autoreload 2
from synthsne import curve_statistics as cs

#methods = ['linear', 'bspline', 'mcmc', 'curvefit']
info_df = cs.get_info_dict(rootdir, methods)
info_df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,trace-time[segs],mb-fit-log-error,mb-n,mb-ok-fits[%],g-fit-log-error,g-ok-fits[%],r-fit-log-error,r-ok-fits[%]
method-linear,.55±.23,3.85±.45,182144,92.829849,3.81±.49,89.950808,3.88±.41,95.708890
method-bspline,.54±.23,5.21±1.99,182144,91.687346,5.21±2.21,88.155525,5.22±1.76,95.219167
method-uniformprior,1.49±.45,7.68±1.10,182144,91.882796,7.76±1.17,88.684777,7.60±1.03,95.080815
method-curvefit,12.77±6.61,3.94±.83,182144,92.835339,3.77±.79,89.953004,4.10±.83,95.717674
method-mcmc,330.13±105.56,3.21±1.02,136448,92.448405,3.02±1.04,90.056285,3.39±.96,94.840525


In [98]:
%load_ext autoreload
%autoreload 2
from flamingchoripan.latex.latex_tables import LatexTable

latex_kwargs = {
    'caption':'???',
    'label':'???',
    'custom_tabular_align':'l|c|cc|cc|cc',
}
latex_table = LatexTable(info_df, **latex_kwargs)
print(latex_table)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\def\srule{\rule{0pt}{2ex}\rule[-1ex]{0pt}{0ex}}
\begin{table*}
\centering
\caption{???}
\label{???}\vspace{.1cm}
\begin{tabular}{l|c|cc|cc|cc}
\hline
method & trace-time[segs] & mb-fit-log-error & mb-n & mb-ok-fits[\%] & g-fit-log-error & g-ok-fits[\%] & r-fit-log-error & r-ok-fits[\%] \srule\\\hline\hline
linear & .55$\pm$.23 & 3.85$\pm$.45 & 182,144 & 92.83 & 3.81$\pm$.49 & 89.95 & 3.88$\pm$.41 & 95.71 \srule\\
bspline & .54$\pm$.23 & 5.21$\pm$1.99 & 182,144 & 91.69 & 5.21$\pm$2.21 & 88.16 & 5.22$\pm$1.76 & 95.22 \srule\\
uniformprior & 1.49$\pm$.45 & 7.68$\pm$1.10 & 182,144 & 91.88 & 7.76$\pm$1.17 & 88.68 & 7.60$\pm$1.03 & 95.08 \srule\\
curvefit & 12.77$\pm$6.61 & 3.94$\pm$.83 & 182,144 & 92.84 & 3.77$\pm$.79 & 89.95 & 4.10$\pm$.83 & 95.72 \srule\\
mcmc & 313.25$\pm$101.11 & 3.23$\pm$1.01 & 107,